In [3]:
import numpy as np
import math

In [4]:
def pred_bayes_logistic(w_map, S_N, theta):
    mu_a = np.dot(w_map.T, theta)
    var_a = np.dot(np.dot(theta.T, S_N), theta)
    kappa_var = (1 + math.pi*var_a/8)^(-0.5)
    x = kappa_var*mu_a
    return 1/(1 + np.exp(-x))

In [5]:
class logistic_sigmoid_model:
    def _init(self):
        self.hyper_para_bayes_logistic = hyper_para_bayes_logistic
        self.pred_bayes_logistic = pred_bayes_logistic
    def training(self, Theta, y):
        self.w0 = np.random.normal(0, 1)
        self.S0 = np.diag(np.random.normal(0, 1, y.shape[0]))
    # Theta n*m (n samples, m features), y n*1
        self.w_map, self.S_N = self.hyper_para_bayes_logistic(self.w0, self.S0,Theta, y)
    def predict(self, theta):
        return self.pred_bayes_logistic(self.w_map, self.S_N, theta)

In [9]:
def multiclass_sigmoid_logistic(Theta, Y):
    n_class = Y.shape[1]
    models = []
    for i in range(n_class):
        models.append(logistic_sigmoid_model())
        models[i].training(Theta, Y[:, i])
    return models